In [1]:
import json

instance = 0

filename = f'train_{instance}.json'

with open('../data/' + filename, 'r') as file:
    data = json.load(file)

depot = data['depot']
patients = data['patients']

with open('../solutions/' + filename, 'r') as file:
    data = json.load(file)

solution = data['phenotype']
fitness = data['fitness']
time_unfitness = data['time_unfitness']
strain_unfitness = data['strain_unfitness']

# Create arcs
visits = []
for route in solution:
    if len(route) == 0:
        continue
    visits.append(0)
    visits.extend(route)
visits.append(0)

arcs = list(zip(visits[:-1], visits[1:]))

def id_2_coords(id):
    if id == 0:
        return depot['x_coord'], depot['y_coord']
    else:
        return patients[str(id)]['x_coord'], patients[str(id)]['y_coord']
    
arcs = [(id_2_coords(a), id_2_coords(b)) for a, b in arcs]

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

depot_x, depot_y = depot['x_coord'], depot['y_coord']
x_coords, y_coords = zip(*[(patient['x_coord'], patient['y_coord']) for patient in patients.values()])



patient_hover_texts = [f"Patient {pid}:<br>Location: ({patient['x_coord']}, {patient['y_coord']})<br>Time Window: [{patient['start_time']}, {patient['end_time']}]" for pid, patient in patients.items()]

# Create a scatter plot for patients
patients_scatter = go.Scatter(x=x_coords, y=y_coords, mode='markers', marker=dict(symbol='circle', color='darkgrey'), name='Patients', text=patient_hover_texts, hoverinfo='text')

# Create a scatter plot for the depot
depot_scatter = go.Scatter(x=[depot_x], y=[depot_y], mode='markers', marker=dict(symbol='square', color='black'), name='Depot')


# Create the figure and add the scatter plots
fig = go.Figure()
fig.add_trace(patients_scatter)
fig.add_trace(depot_scatter)
from itertools import cycle

# Generate a list of distinct colors
num_routes = 25  # Or however many routes you have


colors = px.colors.qualitative.Light24
colors.append('#1CA71C')
colors.append('#FFA500')
colors = iter(cycle(colors))
# Current color

color_ind = 1



color = next(colors)
for start, end in arcs:
    if (start[0] == depot_x and start[1] == depot_y):  # Start at the depot
        color = next(colors)  # Change color for the new route
    fig.add_trace(go.Scatter(x=[start[0], end[0]], y=[start[1], end[1]], mode='lines', line=dict(color=color), legendgroup=f'R{color_ind}'))

# Set axis labels
fig.update_layout(
    xaxis_title='X Coordinate', 
    yaxis_title='Y Coordinate', 
    title=f'Instance {instance} with Fitness={fitness:.2f}, Time Unfitness={time_unfitness:.2f}, Strain Unfitness={strain_unfitness:.2f}', 
    plot_bgcolor='white', 
    paper_bgcolor='white',
    height=600,
)

# Show the plot
fig.show()

In [3]:
for arc in arcs:
    if (2, 40) in arc:
        print(arc)

((0, 40), (2, 40))
((2, 40), (8, 40))


In [4]:
colors = px.colors.qualitative.Light24


In [5]:
colors[0]

'#FD3216'

In [6]:
arcs

[((40, 50), (31, 52)),
 ((31, 52), (26, 52)),
 ((26, 52), (24, 58)),
 ((24, 58), (20, 50)),
 ((20, 50), (2, 60)),
 ((2, 60), (6, 68)),
 ((6, 68), (8, 56)),
 ((8, 56), (15, 60)),
 ((15, 60), (40, 50)),
 ((40, 50), (2, 45)),
 ((2, 45), (5, 45)),
 ((5, 45), (8, 45)),
 ((8, 45), (0, 40)),
 ((0, 40), (2, 40)),
 ((2, 40), (8, 40)),
 ((8, 40), (10, 35)),
 ((10, 35), (10, 40)),
 ((10, 40), (30, 60)),
 ((30, 60), (40, 50)),
 ((40, 50), (27, 43)),
 ((27, 43), (25, 30)),
 ((25, 30), (21, 24)),
 ((21, 24), (30, 25)),
 ((30, 25), (26, 35)),
 ((26, 35), (37, 47)),
 ((37, 47), (40, 50)),
 ((40, 50), (35, 40)),
 ((35, 40), (10, 20)),
 ((10, 20), (5, 5)),
 ((5, 5), (12, 24)),
 ((12, 24), (4, 18)),
 ((4, 18), (0, 45)),
 ((0, 45), (5, 35)),
 ((5, 35), (20, 20)),
 ((20, 20), (15, 10)),
 ((15, 10), (23, 3)),
 ((23, 3), (35, 5)),
 ((35, 5), (42, 5)),
 ((42, 5), (40, 50)),
 ((40, 50), (37, 31)),
 ((37, 31), (38, 5)),
 ((38, 5), (40, 5)),
 ((40, 5), (42, 10)),
 ((42, 10), (42, 12)),
 ((42, 12), (40, 15)),
 ((

In [7]:
data['genotype']

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 4,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 2,
 4,
 5,
 8,
 6,
 3,
 1,
 9,
 2,
 6,
 3,
 4,
 4,
 1,
 11,
 7,
 8,
 6,
 4,
 6,
 7,
 11,
 1,
 10,
 7,
 9,
 1,
 4,
 4,
 6,
 4,
 1,
 1,
 8,
 11,
 3,
 5,
 6,
 6,
 3,
 4,
 1,
 8,
 3,
 5,
 8,
 9,
 7,
 8,
 7,
 4,
 1,
 3,
 10]